# Airbnb Queries

In [64]:
import pandas as pd
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('my_keyspace')

# Query 1:  List available Portland listings for a 2-day period, sorted by rating

In [63]:
query1 = "SELECT id FROM listings WHERE city = 'Portland' ALLOW FILTERING"
portland_listings = session.execute(query1)
portland_listing_ids = [row.id for row in portland_listings]

date_range = ["2025-07-15", "2025-07-16"]

available_listings = []
for listing_id in portland_listing_ids:
    is_available = True
    
    for date_str in date_range:
        query2 = "SELECT available FROM calendar WHERE listing_id = %s AND date = %s"
        result = session.execute(query2, (listing_id, date_str))
        row = result.one()
        if not row or not row.available:
            is_available = False
            break
    
    if is_available:
        available_listings.append(listing_id)

listing_details = []
for listing_id in available_listings:
    query3 = """SELECT id, name, room_type, accommodates, property_type, price, review_scores_rating, city, neighbourhood 
                FROM listings WHERE id = %s"""
    result = session.execute(query3, (listing_id,))
    row = result.one()
    if row:
        listing_details.append({
            'listing_id': row.id,
            'name': row.name,
            'city': row.city,
            'neighborhood': row.neighbourhood,
            'room_type': row.room_type,
            'accommodates': row.accommodates if hasattr(row, 'accommodates') else None,
            'property_type': row.property_type,
            'price': float(row.price) if row.price else 0,
            'review_scores_rating': float(row.review_scores_rating) if row.review_scores_rating else 0
        })

results_df = pd.DataFrame(listing_details)
if not results_df.empty:
    results_df = results_df.sort_values(by='review_scores_rating', ascending=False)
    display(results_df)
else:
    print("Nope")

,listing_id,name,city,neighborhood,room_type,accommodates,property_type,price,review_scores_rating
1383,680519123710389248,Sunny corner w/ open floor plan,Portland,Piedmont,Entire home/apt,4,Entire rental unit,93.0,5.0
1891,1268201952297658624,Luxury downtown condo in the pearl,Portland,Pearl,Entire home/apt,4,Entire rental unit,110.0,5.0
1908,40122072,Charming home in safe beautiful area - fenced ...,Portland,Northwest Heights,Entire home/apt,9,Entire home,300.0,5.0
557,23609556,Bright House-share with Private Bath Near Airport,Portland,Roseway,Private room,4,Private room in home,90.0,5.0
1904,984287915670628608,Play&Work in Southwest Portland,Portland,South Portland,Entire home/apt,4,Entire home,999.0,5.0
...,...,...,...,...,...,...,...,...,...
1589,1208048856128418304,Cozy Mt Tabor Cottage!,Portland,Mt. Tabor,Entire home/apt,2,Entire home,90.0,0.0
272,958750368664910208,Private room in Overlook,Portland,Overlook,Private room,1,Private room in home,60.0,0.0
1581,831129344738314880,Wyndham Portland Waterfront Park Resort | 1BR/1BA,Portland,Portland Downtown,Entire home/apt,4,Entire condo,796.0,0.0
777,1164555353538642176,3 BDR Perfect for Extended Stays Pet Friendly,Portland,Powellhurst-Gilbert,Entire home/apt,6,Entire home,160.0,0.0


# Query 2: Neighborhoods with no listings in a given month

In [ ]:
all_cities = ["Portland", "Los Angeles", "Salem", "San Diego"]

all_neighborhoods = []
for city in all_cities:
    neighborhoods_query = "SELECT neighbourhood FROM neighbourhoods WHERE city = %s"
    neighborhoods = session.execute(neighborhoods_query, (city,))
    city_neighborhoods = [(row.neighbourhood, city) for row in neighborhoods]
    all_neighborhoods.extend(city_neighborhoods)

target_month = "2025-07"

active_neighborhoods = set()
for neighborhood, city in all_neighborhoods:
    query = "SELECT id FROM listings WHERE city = %s AND neighbourhood = %s ALLOW FILTERING"
    neighborhood_listings = session.execute(query, (city, neighborhood))
    neighborhood_listing_ids = [row.id for row in neighborhood_listings]
    
    for listing_id in neighborhood_listing_ids:
        cal_query = "SELECT date FROM calendar WHERE listing_id = %s AND date >= %s AND date < %s AND available = true ALLOW FILTERING"
        start_date = f"{target_month}-01"
        end_date = f"{target_month}-31"
        calendar_entries = session.execute(cal_query, (listing_id, start_date, end_date))
        
        if calendar_entries.one():
            active_neighborhoods.add((neighborhood, city))
            break

inactive_neighborhoods = [(n, c) for n, c in all_neighborhoods if (n, c) not in active_neighborhoods]

inactive_df = pd.DataFrame(inactive_neighborhoods, columns=['neighborhood', 'city'])
if not inactive_df.empty:
    display(inactive_df)
else:
    print("Nope")

Found 4 cities: ['Portland', 'Los Angeles', 'Salem', 'San Diego']


,neighborhood,city
0,Maywood Park,Portland
1,Northwest Industrial,Portland
2,Woodland Park,Portland
3,Bradbury,Los Angeles
4,Hansen Dam,Los Angeles
5,Littlerock,Los Angeles
6,South Diamond Bar,Los Angeles
7,Val Verde,Los Angeles
8,Whittier Narrows,Los Angeles
9,Amphitheater And Water Park,San Diego


# Query 3: Entire-home availability in Salem with bookable chunks

In [ ]:
salem_home_query = "SELECT id, name, minimum_nights FROM listings WHERE city = 'Salem' AND room_type = 'Entire home/apt' ALLOW FILTERING"
salem_homes = session.execute(salem_home_query)
salem_home_listings = [(row.id, row.name, row.minimum_nights) for row in salem_homes]

target_month = "2025-07"
start_date = f"{target_month}-01"
end_date = f"{target_month}-31"
min_chunk_size = 3  

available_chunks = []
for listing_id, name, min_nights in salem_home_listings:
    avail_query = "SELECT date FROM calendar WHERE listing_id = %s AND date >= %s AND date <= %s AND available = true ALLOW FILTERING"
    avail_days = session.execute(avail_query, (listing_id, start_date, end_date))
    
    avail_dates = sorted([str(row.date) for row in avail_days])
    
    if len(avail_dates) < min_chunk_size:
        continue

    chunks = []
    if avail_dates:
        current_chunk = [avail_dates[0]]
        prev_date = avail_dates[0]
        
        for i in range(1, len(avail_dates)):
            current_day = int(avail_dates[i][-2:])
            prev_day = int(prev_date[-2:])
            
            if current_day == prev_day + 1:
                current_chunk.append(avail_dates[i])
            else:
                if len(current_chunk) >= min_chunk_size:
                    chunks.append((current_chunk[0], current_chunk[-1], len(current_chunk)))
                
                current_chunk = [avail_dates[i]]
            
            prev_date = avail_dates[i]
        
        if len(current_chunk) >= min_chunk_size:
            chunks.append((current_chunk[0], current_chunk[-1], len(current_chunk)))
    
    for chunk_start, chunk_end, chunk_length in chunks:
        if chunk_length >= min_nights:
            available_chunks.append({
                'listing_id': listing_id,
                'name': name,
                'chunk_start': chunk_start,
                'chunk_end': chunk_end,
                'nights': chunk_length,
                'min_nights_required': min_nights
            })

result_df = pd.DataFrame(available_chunks)
if not result_df.empty:
    display(result_df)
else:
    print("Nope")

Found 249 entire home listings in Salem


,listing_id,name,chunk_start,chunk_end,nights,min_nights_required
0,48000304,1950''s Charm-Great for long-term/contract/RN/...,2025-07-01,2025-07-17,17,5
1,48000304,1950''s Charm-Great for long-term/contract/RN/...,2025-07-23,2025-07-31,9,5
2,745182968152467328,Unit C at the Manor House (month to month),2025-07-01,2025-07-31,31,29
3,50447645,"Jefferson Garden Cottage, Hosted by Salemstays...",2025-07-01,2025-07-17,17,2
4,50447645,"Jefferson Garden Cottage, Hosted by Salemstays...",2025-07-21,2025-07-31,11,2
...,...,...,...,...,...,...
240,32018098,Oregon Winemakers Beautiful Colonial in Salem,2025-07-22,2025-07-31,10,2
241,785111814066982784,Scout- a rustic “cabin” apartment in the city,2025-07-01,2025-07-31,31,1
242,1151715665539943936,49th Ave Tiny House,2025-07-01,2025-07-16,16,2
243,1151715665539943936,49th Ave Tiny House,2025-07-22,2025-07-31,10,2
